Uncertainty analysis for flight sim:
1. Creates a normal distribution for each parameter of interest using an assumed mean and std gathered from REs/testing
2. Samples each parameter from the distribution 100k times and runs the flight sim code using those params to establish a confidence interval for apogee

In [1]:
# !pip install nbimporter
# import nbimporter
# from ALULA_ThrustFlight_v2 import thrust_sim
!pip install rocketcea
from thrust_func import thrust_sim

import numpy as np
import rocketcea
import os
import numpy as np
import scipy
from scipy.optimize import minimize
import tkinter as tk
import matplotlib.pyplot as plt
import matplotlib
print(matplotlib.matplotlib_fname())
from rocketcea.cea_obj import CEA_Obj
import pandas as pd 
import math
import datetime
from numpy.random import normal, uniform, choice
from IPython.display import display
import matplotlib.pyplot as plt

/opt/conda/lib/python3.9/site-packages/matplotlib/mpl-data/matplotlibrc


Means for parameters of interest defined below

In [2]:
#Pressure
from rocketcea.cea_obj import CEA_Obj
from thrust_func import *
P_ox_mu = 505.0 #from AlULA_ThrustFlight (psi)
P_eth_mu = 495.0 #""

#Tank Fill
V_ox_mu = 3.85 #from AlULA_ThrustFlight (L)
V_eth_mu = 2.15 #""

#Hydraulic Resistances
CdA_ox_inj_mu = 0.00001323 #probs calculate using excel -- values seem very off from spreadsheet????
CdA_eth_inj_mu = 0.000009061

CdA_ox_feed_mu = 0.000040 #Test For Cd NEEDS DATA
CdA_eth_feed_mu = 0.00004046 #Test For Cd

#Cstar Efficiency
Efficiency_mu = 0.91

#General Distribution/Sampling Parameters (assumes Gaussian for everything)
N_samples = 100000 
results = []
#Pressure
P_sd = 0.02

#Tank Fill
# Fill_ox_sd = 0.0255 / V_oxinit #(L/L) ... taken from 0.051 = 2sigma fill out properly later
V_ox_sd = 0.02
V_eth_sd = 0.02

#Hydraulic Resistances
CdA_ox_inj_sd = 0.02 #probs calculate using excel -- values seem very off from spreadsheet????
CdA_eth_inj_sd = 0.02

CdA_ox_feed_sd = 0.02
CdA_eth_feed_sd = 0.02

#Cstar Efficiency
Efficiency_sd = 0.02

Sampling code:

In [ ]:
import numpy as np
for _ in range(N_samples):
    sampled_P_ox = np.random.normal(P_ox_mu, P_sd * P_ox_mu)
    sampled_P_eth = np.random.normal(P_eth_mu, P_sd * P_eth_mu)
    
    sampled_V_ox = np.random.normal(V_ox_mu, V_ox_sd * V_ox_mu)
    sampled_V_eth = np.random.normal(V_eth_mu, V_eth_sd * V_eth_mu)
    
    sampled_CdA_inj_ox = np.random.normal(CdA_ox_inj_mu, CdA_ox_inj_sd * CdA_ox_inj_mu)  # Assuming you have CdA_inj_LOX defined
    sampled_CdA_inj_eth = np.random.normal(CdA_eth_inj_mu, CdA_eth_inj_sd * CdA_eth_inj_mu)  # Assuming you have CdA_inj_ETH defined
    
    sampled_CdA_feed_ox = np.random.normal(CdA_ox_feed_mu, CdA_ox_feed_sd * CdA_ox_feed_mu)  # Assuming you have CdA_inj_LOX defined
    sampled_CdA_feed_eth = np.random.normal(CdA_eth_feed_mu, CdA_eth_feed_sd * CdA_eth_feed_mu)  # Assuming you have CdA_inj_ETH defined
    
    sampled_Efficiency = np.random.normal(Efficiency_mu, Efficiency_sd)
    chamber = CEA_Obj(propName="", oxName="LOX", fuelName="C2H5OH")
    
    result = thrust_sim(sampled_P_ox, sampled_P_eth, sampled_V_ox, sampled_V_eth, sampled_CdA_inj_ox, sampled_CdA_inj_eth, sampled_CdA_feed_ox, sampled_CdA_feed_eth, sampled_Efficiency, chamber)
    results.append(result)

Plotting histogram based off varying parameter